### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/spscientist/students-performance-in-exams
2. Соберем пайплайн с препроцессингом 
3. Обучим модель и сохраним на диск предобученный пайплайн

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets
from sklearn.preprocessing import StandardScaler

Загрузим данные

In [2]:
df = pd.read_csv("./StudentsPerformance.csv.xls")
df.head(400)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
395,male,group A,high school,free/reduced,none,48,45,41
396,female,group B,high school,free/reduced,none,50,67,63
397,female,group C,associate's degree,standard,none,85,89,95
398,male,group B,some high school,standard,none,74,63,57


In [3]:
df['test preparation course'].value_counts()

none         642
completed    358
Name: test preparation course, dtype: int64

In [4]:
df.loc[df['math score']==0, 'math score'] = 1

In [5]:
dct = {'female': 1, 'male': 0}
df['gender'] = df['gender'].map(dct)

In [6]:
df['gender'].value_counts(dropna=False)

1    518
0    482
Name: gender, dtype: int64

In [7]:
df.rename(columns={'race/ethnicity': 'race_ethnicity', 
                   'parental level of education': 'parental_level_of_education',
                   'test preparation course': 'test_preparation_course',
                   'math score': 'math_score',
                   'reading score': 'reading_score',
                   'writing score': 'writing_score'
                  }, inplace=True)

In [8]:
df.head(3)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,1,group B,bachelor's degree,standard,none,72,72,74
1,1,group C,some college,standard,completed,69,90,88
2,1,group B,master's degree,standard,none,90,95,93


Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('gender', 1),  
                                                    df['gender'], test_size=0.33, random_state=42)

#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [10]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

К полям:
race/ethnicity, parental level of education, lunch, test preparation course  применим OHE-кодирование
math score, reading score, writing score - standardScaler

In [11]:
X_train.columns

Index(['race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'math_score', 'reading_score',
       'writing_score'],
      dtype='object')

Соберем кусок, ответственный за feature engineering

In [12]:
continuos_cols = ['math_score', 'reading_score', 'writing_score']
cat_cols = ['race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course']

continuos_transformers = []
cat_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))


Теперь объединим все наши трансформеры с помощью FeatureUnion

In [13]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-0.24218231, -0.40097854, -0.10836734, ...,  1.        ,
         0.        ,  1.        ],
       [-0.04103089, -0.68310573, -0.7881999 , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.49537291,  1.43284823,  1.31928104, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.91268706, -0.54204214, -1.06013293, ...,  1.        ,
         0.        ,  1.        ],
       [-1.11383848, -1.52948732, -1.06013293, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.23292813,  1.50338003,  1.3872643 , ...,  1.        ,
         0.        ,  1.        ]])

Добавим простейший классификатор

In [14]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

pipeline.fit(X_train, y_train)

CPU times: user 45.2 ms, sys: 3.81 ms, total: 49.1 ms
Wall time: 41.4 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('math_score',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='math_score')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('reading_score',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='reading_score')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('writing_score',
                                                 Pipeline(steps=[('selector',
     

Посмотрим, как выглядит наш pipeline

In [15]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('math_score',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='math_score')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('reading_score',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='reading_score')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('writing_score',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='writing_score')),
                                                  ('standard',
                                                

Сохраним модель (пайплайн)

In [16]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)